<a href="https://colab.research.google.com/github/ericson-j-santos/Engenharia-de-Prompt-criando-prompts-eficazes-para-IA-Generativa/blob/main/Curso_Engenharia_de_Prompt_Self_consistency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q openai

Depois, importamos a biblioteca e precisamos de uma API para nos identificarmos para a OpenAI. Nós usamos uma API key, que é um valor que identifica quem somos para a OpenAI.

Temos a nossa mensagem inicial, que define o papel do modelo. Neste caso, o modelo é um assistente prestativo. Depois, definimos os parâmetros iniciais do ChatGPT, que é o valor mais básico da maioria dos modelos.

In [ ]:
from google.colab import userdata
import openai

# seta a API key
openai.api_key = userdata.get('OPENAI_API_KEY')

# Definição de persona do sistema e histórico de mensagens
messages = [
    {"role": "system", "content": "Você é um assistente prestativo."},  # seta a persona do modelo, se vai ser educado, formal, sarcástico etc.
]

A parte interessante começa quando criamos uma variável chamada answers, que inicialmente está vazia. Vamos supor que faremos a pergunta quatro vezes, então o número de loopings será quatro.

In [ ]:
# SELF-CONSISTENCY
answers = []

LOOPS = 4

prompt = """Retorne no final sempre a string 'Resultado: ' com apenas valor do resultado depois dela, nenhuma operação nem pontuação além do valor.

Pergunta: Se houver 3 carros no estacionamento e chegarem mais 2 carros, quantos carros estarão no estacionamento?

Resposta: Já existem 3 carros no estacionamento. Chegam mais 2. Agora existem 3 + 2 = 5 carros. A resposta é 5.

Resultado: 5

Pergunta: Os patos de Janet botam 16 ovos por dia. Ela come três no café da manhã todas as manhãs e faz muffins para seus amigos todos os dias utilizando mais quatro. Ela vende o restante por 2 dólares por ovo. Quanto dinheiro ela ganha todos os dias?

Resposta:

Resultado:"""

Estamos fazendo a nossa cadeia de pensamento e ensinando o modelo a raciocinar com o exemplo dos carros. A pergunta, a resposta e o resultado do problema dos carros somos nós que estamos fornecendo.

Logo abaixo, está o caso real, para o qual queremos uma resposta. Se a Janet tinha 16 ovos, comeu três e usou outros quatro para fazer muffins, sobraram nove. Assim, multiplicando nove ovos por dois dólares, temos 18 dólares.

Deixamos a resposta e o resultado em branco para o modelo preencher, seguindo o nosso padrão de Chain of Thoughts. Depois de definir o prompt, colocamos ele dentro de uma variável e essa variável será enviada dentro das mensagens para a OpenAI.

Fazemos um for loop, que é basicamente uma repetição. Fazemos uma vez tudo que está dentro do looping, depois fazemos uma segunda vez, depois uma terceira vez, até chegar no número que pedimos para parar.

Fazemos a chamada para a OpenAI, pegamos a resposta e adicionamos na lista de respostas (answers). No final, queremos ver qual é o valor mais frequente e quantas vezes ele aparece.

In [ ]:
messages.append({"role": "user", "content": prompt})  # adiciona sua pergunta no histórico do chat

for loop in range(0, LOOPS):
    # Chamada à API
    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = messages,
        max_tokens = 200,
        temperature = 1
    )

    # Extrai a resposta de dentro do JSON de resposta
    answer = response.choices[0].message.content
    print(f"### LOOP", loop, ':\n')
    print(answer)

    start_index = answer.find("Resultado: ") + len("Resultado: ")
    if start_index != -1:
        number_answer = answer[start_index:].split()[0].rstrip('.')
        print("Answer:", number_answer, '\n------')
        answers.append(int(number_answer))  # adiciona a resposta à lista de respostas

print(answers, '\n')

Vamos rodar o código e ver como ele funciona na prática. Rodamos o código e vemos que o valor mais frequente é 18, que aparece quatro vezes.

Vamos executar mais uma vez o mesmo código. Desta vez, o valor mais frequente é 18, que aparece três vezes. Então, entendemos como funciona a técnica. Poderíamos fazer mais vezes, por exemplo, 10 ou 15 vezes, e pegar o valor mais frequente, se for algo mais importante

In [ ]:
[18, 18, 24, 18]



O valor mais frequente é 18 que aparece 3 vezes.



Baseado na ideia de que a maioria das vezes o modelo estará correto, conseguimos obter o resultado. No entanto, esse código não está perfeito, porque às vezes o modelo não retorna exatamente o resultado de dois pontos, ele retorna um texto, e, nessa conversão, se ele pegar um texto vai dar um erro, e não poder comparar texto com número.

Mas já serve para explicar como funciona essa técnica de Self-Consistency. Esse código base estará disponível para vocês melhorarem e adaptarem para suas necessidades.